<a href="https://colab.research.google.com/github/pankaj-juneja/dask/blob/master/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
import datasets
import torch
import pandas as pd
import random
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
import os

def generate_sample_data(output_file="sample_data.csv", num_samples=100):
    texts = [
        "The Federal Reserve announced an interest rate hike to combat inflation.",
        "Apple Inc. reported a record quarterly revenue of $123.9 billion.",
        "Tesla's stock price surged after strong vehicle delivery numbers.",
        "Crude oil prices fell as global supply concerns eased.",
        "Bitcoin dropped below $40,000 amid market uncertainty."
    ]
    summaries = [
        "Fed raises interest rates to curb inflation.",
        "Apple posts record revenue of $123.9B.",
        "Tesla stock jumps on strong deliveries.",
        "Oil prices decline as supply stabilizes.",
        "Bitcoin dips below $40K amid volatility."
    ]

    data = {"text": [], "summary": []}
    for _ in range(num_samples):
        idx = random.randint(0, len(texts) - 1)
        data["text"].append(texts[idx])
        data["summary"].append(summaries[idx])

    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    print(f"Sample data generated and saved to {output_file}")

def load_data(dataset_path):
    dataset = load_dataset("csv", data_files=dataset_path)
    return dataset["train"].train_test_split(test_size=0.1)

def preprocess_data(examples, tokenizer, max_length=1024):
    model_inputs = tokenizer(examples["text"], max_length=max_length, truncation=True, padding="max_length")
    labels = tokenizer(examples["summary"], max_length=200, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels
    return model_inputs

def train_model(dataset_path, model_name="facebook/bart-large-cnn", output_dir="finetuned_model"):
    dataset = load_data(dataset_path)
    tokenizer = BartTokenizer.from_pretrained(model_name)
    dataset = dataset.map(lambda x: preprocess_data(x, tokenizer), batched=True)

    model = BartForConditionalGeneration.from_pretrained(model_name)
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=2,
        fp16=torch.cuda.is_available()
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
    )

    trainer.train()
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)

def summarize_text(text, model_path="finetuned_model"):
    tokenizer = BartTokenizer.from_pretrained(model_path)
    model = BartForConditionalGeneration.from_pretrained(model_path)
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=200, min_length=50, do_sample=False)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def main():
    sample_data_path = "sample_data.csv"
    generate_sample_data(sample_data_path)
    train_model(sample_data_path)

    test_text = "The Federal Reserve increased interest rates to manage inflation risks."
    summary = summarize_text(test_text)
    print(f"Generated Summary: {summary}")

if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.1 MB/s eta 0:00:00
Sample data generated and saved to sample_data.csv


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-86df5aab1c95>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

# New Section